In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# Changing working directory
import os

os.chdir('/content/drive/My Drive/QA-tasks-ML/github_codes/qaml/')

Mounted at /content/drive


In [1]:
# %pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0  \
    evaluate==0.4.0  \
    rouge_score==0.1.2  \
    loralib==0.1.1  \
    peft==0.3.0   \
    mistletoe --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 78.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torchdata==0.6.1, but you have torchdata 0.5.1 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have

In [2]:
from datasets import load_dataset, Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
# from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import re

In [4]:
!ls resources

 docs-db				     SampleQuestions.xlsx
'KnowledgeDocument(pan_card_services).txt'   WriteupTemplate.docx


In [5]:
line = '***Is it mandatory to link Aadhaar with PAN for NRI?***'
if re.search("^(\*\*)(.*\*\*)$", line) is not None:
    print(f'line : {line}')
else:
    print(f'Nothing ....')

line : ***Is it mandatory to link Aadhaar with PAN for NRI?***


In [3]:
# parsing markdown contents

def parse_markdown_contents(filepath:str):
    parsed_contents = {}

    ques_list = []
    ans_list = []
    tmp_ans_buffer = ""
    ques_bool = False

    fp = open(filepath, 'r')
    lines = fp.readlines()

    for line_no, line in enumerate(lines):
        line = line.strip()
        # print(f'line no : {line_no}')
        # print(f'line_no : {line_no} :: contents : {line}')
        if line.startswith('# ') or line.startswith('## ') or line == '':
            continue
        if line.startswith('###') or ((re.search("^(\*\*)(.*\*\*)$", line) is not None) and (line_no >= 213) ):
            ques_line = line.replace('###', '').replace('**','')
            ques_list.append(ques_line)
            if tmp_ans_buffer != '':
                ans_list.append(tmp_ans_buffer)
                tmp_ans_buffer = ''
        elif re.search("^[a-zA-z0-9]", line) is not None:
            tmp_ans_buffer +=  line + '\n'
        elif len(ques_list) > len(ans_list) and re.search("^([*]|[-])", line) is not None:
            tmp_ans_buffer += line + '\n'

        # if line_no == 33:
        #     print(f'line : {line} :: ques_bool : {ques_bool}')
        #     print(f'condition check : {line.startswith("**")}')
        # if line_no >= 314: # 38;56
        #     print(f'line no : {line_no}')
        #     ans_list.append(tmp_ans_buffer)
        #     break
    ans_list.append(tmp_ans_buffer)

    return ques_list, ans_list







In [4]:
qna_custom_file = './resources/KnowledgeDocument(pan_card_services).txt'

ques_list, ans_list = parse_markdown_contents(qna_custom_file)

print(f'len ques_list : {len(ques_list)} :: len ans_list : {len(ans_list)}')

indx = 0

print(f'ques :\n {ques_list[indx]}\n :: ans : \n{ans_list[indx]}')

print('-'*20)

print(f'ques :\n {ques_list[len(ques_list)-1]}\n :: ans : \n{ans_list[len(ans_list)-1]}')

FileNotFoundError: ignored

In [8]:
df = pd.DataFrame({'Question': ques_list, 'Answers': ans_list})

In [9]:
df.tail()

,Question,Answers
43,Can NRIs use an international card for making ...,"Yes, NRIs can use their international debit or..."
44,Is it necessary to download the ABC app or vis...,"No, it is not necessary to download the ABC ap..."
45,Can I apply for pan card without Aadhaar?,"Yes, NRIs can apply for a PAN card without an ..."
46,Can I apply for a PAN card if I am a non-resid...,"Yes, as an NRI, you can apply for a PAN card. ..."
47,Can I take the delivery of Pan card at Indian ...,"Yes, you can take the delivery of your PAN car..."


In [10]:
indx = 24

print(f'question is : {ques_list[indx]}')
print('-'*20)
print(f'Ans is : \n{ans_list[indx]}')

question is : Can I apply for pan card from USA?
--------------------
Ans is : 
Yes. You can apply for a PAN Card from the USA. The easiest and most convenient way to apply for a PAN card from the USA is through ABC.



In [15]:
# converting 'dataframe' to 'DataSet'(huggingface)
datasets = Dataset.from_pandas(df).train_test_split(test_size=0.2)
datasets

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answers'],
        num_rows: 38
    })
    test: Dataset({
        features: ['Question', 'Answers'],
        num_rows: 10
    })
})

In [13]:
# for question answers
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# qa_model_name = 'distilbert-base-uncased'
# qa_orig_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)
# qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
# if 'cuda' in device:
#     qa_orig_model = qa_orig_model.to(device)

# print(f'Device chosen is : {device}')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

Device chosen is : cuda:0


In [11]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_name = 'google/flan-t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)#, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# assigning to GPU
if "cuda" in device:
  # print(f'Putting model in CUDA.')
  original_model = original_model.to(device)

print(f'Device chosen is : {device}')

Device chosen is : cuda:0


In [13]:
def print_number_of_trainable_parameters(model):
  trainable_model_params = 0
  all_model_params = 0
  for _,param in model.named_parameters():
    all_model_params += param.numel()
    if param.requires_grad:
      trainable_model_params += param.numel()
  print(f'Trainable model parameters : {trainable_model_params} \nAll model parameters : {all_model_params}')


print_number_of_trainable_parameters(original_model)
# print_number_of_trainable_parameters(qa_orig_model)

Trainable model parameters : 247577856 
All model parameters : 247577856


In [17]:
def tokenize_function(example, tokenizer):
    questions = [ques for ques in example['Question']]
    example['input_ids'] = tokenizer(questions, padding='max_length', truncation=True, return_tensors='pt').input_ids.to(device)
    example['labels'] = tokenizer(example['Answers'], padding='max_length', truncation=True, return_tensors='pt').input_ids.to(device)

    return example

tokenized_datasets = datasets.map(tokenize_function, batched=True, fn_kwargs={"tokenizer": tokenizer})
# tokenized_datasets = datasets.map(tokenize_function, batched=True, fn_kwargs={"tokenizer": qa_tokenizer})
tokenized_datasets = tokenized_datasets.remove_columns(['Question', 'Answers',])

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [18]:
tokenized_datasets.shape

{'train': (38, 2), 'test': (10, 2)}

In [19]:
tokenized_datasets.keys()

dict_keys(['train', 'test'])

In [21]:
output_dir = f'./checkpoints/qa-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir = output_dir,
    learning_rate = 1e-5,
    num_train_epochs = 100,
    weight_decay = 0.01,
    logging_steps = 10,
    max_steps = 100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16
)

trainer = Trainer(
    model = original_model,
    # model = qa_orig_model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    # eval_dataset = tokenized_datasets['test']
)

In [22]:
trainer.train()

OutOfMemoryError: ignored

root          21      19  0 17:09 ?        00:00:00 /usr/colab/bin/kernel_manager_proxy --listen_port=6000 --target_port=9000 --logtostderr --listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https://colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-2vvry3rxwgtkp --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=true
root        4512    1074  0 17:25 ?        00:00:00 /bin/bash -c ps -ef | grep 'gpu'
root        4514    4512  0 17:25 ?        00:00:00 grep gpu


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
from torch.utils.data import DataLoader
from datasets import Dataset, load_dataset
from transformers import TrainingArguments, Trainer
from tqdm import tqdm

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Define your tokenize_function
def tokenize_function(example, tokenizer):
    questions = [ques for ques in example['Question']]
    example['input_ids'] = tokenizer(questions, padding='max_length', truncation=True, return_tensors='pt').input_ids.to(device)
    example['labels'] = tokenizer(example['Answers'], padding='max_length', truncation=True, return_tensors='pt').input_ids.to(device)

    return example

# Example data (replace with your actual data)
questions = [
    "What is the capital of France?",
    "Who is the author of Harry Potter?"
]

answers = [
    "The capital of France is Paris.",
    "The author of Harry Potter is J.K. Rowling."
]

# questions = ques_list[0:5]
# answers = ans_list[0:5]

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')

# Set up the training data using datasets library
train_dataset = Dataset.from_dict({'Question': questions, 'Answers': answers})

# Preprocess the dataset using the tokenize_function
train_dataset = train_dataset.map(tokenize_function, batched=True, fn_kwargs={"tokenizer": tokenizer})
train_dataset.set_format(type='torch', columns=['input_ids', 'labels'])

# Set up the data loader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Set up the optimizer and training parameters
optimizer = AdamW(model.parameters(), lr=2e-5)

# Fine-tuning the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.train()

# Training arguments
output_dir = 'output_directory'
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=100,
    weight_decay=0.01,
    logging_steps=10,
    max_steps=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16
)

# Create the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model
# trainer.save_model(output_dir)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,38.962200
20,30.913600
30,26.218500
40,22.300100
50,18.940900
60,15.676000
70,12.511900
80,10.043200
90,8.047800
100,7.118400


TrainOutput(global_step=100, training_loss=19.073257751464844, metrics={'train_runtime': 70.379, 'train_samples_per_second': 22.734, 'train_steps_per_second': 1.421, 'total_flos': 136951470489600.0, 'train_loss': 19.073257751464844, 'epoch': 100.0})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW
from torch.utils.data import DataLoader
from datasets import Dataset, load_dataset
from transformers import TrainingArguments, Trainer
from tqdm import tqdm

# Define your tokenize_function
def tokenize_function(example, tokenizer):
    questions = [ques for ques in example['Question']]
    example['input_ids'] = tokenizer(questions, padding='max_length', truncation=True, return_tensors='pt').input_ids.to(device)
    example['labels'] = tokenizer(example['Answers'], padding='max_length', truncation=True, return_tensors='pt').input_ids.to(device)

    return example

# Example data (replace with your actual data)
questions = [
    "What is the capital of France?",
    "Who is the author of Harry Potter?"
]

answers = [
    "The capital of France is Paris.",
    "The author of Harry Potter is J.K. Rowling."
]

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-base')
model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')

# Set up the training data using datasets library
train_dataset = Dataset.from_dict({'Question': questions, 'Answers': answers})

# Preprocess the dataset using the tokenize_function
train_dataset = train_dataset.map(tokenize_function, batched=True, fn_kwargs={"tokenizer": tokenizer})
train_dataset.set_format(type='torch', columns=['input_ids', 'labels'])

# Set up the data loader
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Set up the optimizer and training parameters
optimizer = AdamW(model.parameters(), lr=2e-5)

# Fine-tuning the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.train()

# Training arguments
output_dir = 'output_directory'
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=100,
    weight_decay=0.01,
    logging_steps=10,
    max_steps=100,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16
)

# Create the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model(output_dir)
